### #1 Data exploration

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Importing all dedicated libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
#from scipy.stats import shapiro
import scipy.stats as st
from scipy import stats
import re
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import train_test_split
#from sklearn import linear_model
#from sklearn.metrics import r2_score
#from sklearn.metrics import mean_squared_error
#import math
#from sklearn.preprocessing import PowerTransformer
#import pickle

In [ ]:
# Print whole df if necessary

pd.set_option('display.max_rows', 100)

In [ ]:
# Reading files into a DF

df = pd.read_csv('./files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')

In [ ]:
# Checking the df

df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Renaming columns
#df.rename(columns=lambda x: x.replace(' ', '_').lower())

df.columns= df.columns.str.lower().str.replace(' ','_')

In [ ]:
df['effective_to_date'].dtype

In [ ]:
# Fixing datetime col

df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

In [ ]:
df['effective_to_date']

In [ ]:
# Checking nan values

df.isnull().sum()

In [ ]:
# Checking duplicated rows

df.duplicated().sum()

In [ ]:
df.columns

In [ ]:
# organizing df ... target variable to the last

df= df[['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel',
       'vehicle_class', 'vehicle_size', 'total_claim_amount']]

In [ ]:
for x in df.columns:
    print( x, '|', 'uniques:', len(df[x].unique()), '|', 'type:', df[x].dtypes)

In [ ]:
numerical_cols = df.select_dtypes(np.number).columns
categorical_cols = df.select_dtypes(np.object).columns
discrete_cols= []
num_cols = []

In [ ]:
# Function to return cat/num/dis variables

def num_dis_cat(data, tipo= ['cat', 'dis', 'num']):
        
    if tipo == 'dis':
        for x in numerical_cols:
            if len(data[x].unique()) < 10:
                discrete_cols.append(x)
        return data[discrete_cols]
    
    elif tipo == 'num':
        for x in numerical_cols:
            if len(data[x].unique()) > 10:
                num_cols.append(x)
        return data[num_cols]
        
    else: return data[categorical_cols]

In [ ]:
discretes= num_dis_cat(df, 'dis')
numericals= num_dis_cat(df, 'num')
categoricals= num_dis_cat(df, 'cat')

In [ ]:
discretes.head()
discretes.shape
discretes['number_of_open_complaints'].value_counts()
discretes['number_of_policies'].value_counts()

In [ ]:
# How can i distribute this variables ??? 

In [ ]:
numericals.head()
numericals.shape

In [ ]:
categoricals.head()
categoricals.shape

### Data engineering every single categorical variable. 

In [ ]:
categoricals['customer'].unique()

In [ ]:
categoricals['state'].value_counts()/len(categoricals['state'])

In [ ]:
sns.countplot(data= categoricals, x= 'state')

In [ ]:
categoricals['state'].unique()

In [ ]:
# grouping non relevant states

rel_states= ['California', 'Oregon']

def grouping_states(x):
    
    if not x in rel_states:
        x= 'Other'
        return x
    else: return x
    
categoricals['state']= categoricals['state'].apply(grouping_states)

In [ ]:
categoricals['state']= categoricals['state'].replace('California state', 'California')

In [ ]:
categoricals['state'].value_counts()

In [ ]:
sns.countplot(data= categoricals, x= 'state')

In [ ]:
categoricals['response'].value_counts()

In [ ]:
categoricals['coverage'].value_counts()

In [ ]:
sns.countplot(data= categoricals, x= 'education')

In [ ]:
categoricals['education'].value_counts()

In [ ]:
# grouping non relevant states

higher_ed= ['Master', 'Doctor']

def grouping_education(x):
    
    if x in higher_ed:
        x= 'Higher Education'
        return x
    else: return x
    
categoricals['education']= categoricals['education'].apply(grouping_education)

In [ ]:
categoricals['employmentstatus'].value_counts()

In [ ]:
categoricals['gender'].value_counts()

In [ ]:
categoricals['location_code'].value_counts()

In [ ]:
categoricals['marital_status'].value_counts()

In [ ]:
categoricals['policy_type'].value_counts()/len(categoricals['policy_type'])

In [ ]:
# we can assume this 2 columns are pretty much the same

categoricals['policy'].value_counts()

In [ ]:
# cleaning policy

def cleaning_pol(x):
    
    if 'Personal' in x:
        x= 'Personal'
        return x
    else: 
        x= 'Other'
        return x
    
categoricals['policy']= categoricals['policy'].apply(cleaning_pol)

In [ ]:
sns.countplot(x= 'policy', data= categoricals)

In [ ]:
# deleting one of the 2 columns

categoricals.drop(columns= 'policy_type', inplace= True)

In [ ]:
categoricals['renew_offer_type'].value_counts()

In [ ]:
# cleaning offers

def cleaning_offer(x):
    
    if '1' in x:
        x= re.sub('Offer', '',x)
        return x

    elif '2' in x:
        x= re.sub('Offer', '',x)
        return x
        
    else: 
        x= re.sub('Offer3|Offer4', '3',x)
        return x
    
categoricals['renew_offer_type']= categoricals['renew_offer_type'].apply(cleaning_offer)

In [ ]:
categoricals['sales_channel']= categoricals['sales_channel'].replace(['Web', 'Call Center'], 'Other')

In [ ]:
categoricals['sales_channel'].value_counts()

In [ ]:
def cleaning_vecl(x):
    
    if ('SUV' or 'Sport') in x:
        x= 'SUV'
        return x
        
    else: 
        x= 'Standard'
        return x

categoricals['vehicle_class']= categoricals['vehicle_class'].apply(cleaning_vecl)

In [ ]:
categoricals['vehicle_class'].value_counts()

In [ ]:
categoricals['vehicle_size'].value_counts()

In [ ]:
categoricals.shape
categoricals.columns

In [ ]:
categoricals.head()

In [ ]:
# checking dependecy btween variables next to each other

for x in range(len(categoricals.columns)-1):
    cross = pd.crosstab(categoricals[categoricals.columns[x]], categoricals[categoricals.columns[x+1]], margins = False)
    P = st.chi2_contingency(cross)
    print('P value of ',categoricals.columns[x], 'and', categoricals.columns[x+1] , P[1])

In [ ]:
# checking the correlation btween certain variables

# state location_code (not dependant)
crosstab1 = pd.crosstab(categoricals['state'], categoricals['location_code'], margins = False)
Pval1= st.chi2_contingency(crosstab1)[1]
print('state | location_code', Pval1)
# not dependant

#sales_channel renwe_offer_type (very dependant)
crosstab2 = pd.crosstab(categoricals['sales_channel'], categoricals['renew_offer_type'], margins = False)
Pval2= st.chi2_contingency(crosstab2)[1]
print('sales_channel | renew_offer_type', Pval2)
# dependant

#sales_channel renwe_offer_type (very dependant)
crosstab3 = pd.crosstab(categoricals['vehicle_class'], categoricals['vehicle_size'], margins = False)
Pval3= st.chi2_contingency(crosstab3)[1]
print('vehicle_class | vehicle_size', Pval3)
# slightly dependant


#marital_status employment_status (very dependant)
crosstab4 = pd.crosstab(categoricals['employmentstatus'], categoricals['marital_status'], margins = False)
Pval4= st.chi2_contingency(crosstab4)[1]
print('employmentstatus | marital_status', Pval4)
# slightly dependant


#renewoffer maritalstatus (very dependant)
crosstab4 = pd.crosstab(categoricals['employmentstatus'], categoricals['marital_status'], margins = False)
Pval4= st.chi2_contingency(crosstab4)[1]
print('employmentstatus | marital_status', Pval4)
# slightly dependant

### From previous analysis, this is the dataset (categoricals only) that will be used from now on

In [ ]:
categoricals_red= categoricals[['gender', 'coverage', 'employmentstatus', 'location_code', 'policy', 'renew_offer_type', 'vehicle_size']]

In [ ]:
categoricals_red.head()

In [ ]:
# Sizes: Small-> S, Medium-> M, Large-> L.

categoricals_red['vehicle_size'].replace({'Medsize':'M', 'Small': 'S', 'Large':'L'}, inplace= True)

In [ ]:
# Policies: Personal-> P, Other-> O

categoricals_red['policy'].replace({'Personal':'P', 'Other': 'O'}, inplace= True)

In [ ]:
# Policies: Suburban-> S, Rural-> R, Urban-> U

categoricals_red['location_code'].replace({'Suburban':'S', 'Rural': 'R', 'Urban': 'U'}, inplace= True)

In [ ]:
categoricals_red['location_code'].unique()

In [ ]:
# empstatus: Suburban-> S, Rural-> R, Urban-> U

#categoricals_red['location_code'].replace({'Suburban':'S', 'Rural': 'R', 'Urban': 'U'}, inplace= True)

In [ ]:
categoricals_red['employmentstatus'].unique()

In [ ]:
# grouping status

emp_status= ['Unemployed', 'Medical Leave', 'Disabled', 'Retired']

def grouping_status(x):
    
    if x in emp_status:
        x= 'O'
        return x
    else: return 'E'
    
categoricals_red['employmentstatus']= categoricals_red['employmentstatus'].apply(grouping_status)

In [ ]:
grouping_status('Unemployed')

In [ ]:
# Coverage: Basic-> B, Premium / Extended-> E

categoricals_red['coverage'].replace({'Extended':'E', 'Premium': 'E', 'Basic': 'B'}, inplace= True)

In [ ]:
categoricals_red['coverage'].unique()

In [ ]:
for x in categoricals_red.columns:
    plt.show()
    print(sns.countplot(x= x, data= categoricals_red))

In [ ]:
# now working on the discrete variables and adding them to the categorical tables

discretes.head()

for x in discretes.columns:
    print (discretes[x].unique())
    print (discretes[x].value_counts())

In [ ]:
# checking p-value
crosstab5 = pd.crosstab(discretes['number_of_open_complaints'], discretes['number_of_policies'], margins = False)
Pval5= st.chi2_contingency(crosstab5)[1]
print('number_of_open_complaints | number_of_policies', Pval5)
#both discretes varaibles are very dependant on each other, lets keep one

In [ ]:
# putting together teh # of complaints in 0 or 1

def complaints(x):

    if x > 1:
        return 2
    else: return x
    
discretes['number_of_policies']= discretes['number_of_policies'].apply(complaints)

In [ ]:
discretes_red = discretes[['number_of_policies']]

In [ ]:
categoricals_red.head(2)
discretes_red.head(2)

In [ ]:
cat_dis = pd.concat([categoricals_red, discretes_red], axis= 1)

In [ ]:
cat_dis.head()

### Data engineering every numericals. 


- Did i properly cleaned up the categorical vars? Does my analysis make any sense?
- How to treat the categoricals i cleaned up

- Trabajar las variables solo con logs y remover outliers antes de hacer el split.. una vez aplicado el split entonces tocar el minmax y el standard scaler
- Que hacer con los outliers??? Quitarlos? la mean? que toeria usar? 